In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import torch
from data import get_midi_data, MidiLoader
from handcrafted import HandcraftedKeyModel
import matplotlib.pyplot as plt
from torch.autograd import Variable

handcrafted = HandcraftedKeyModel()
midi_data = get_midi_data(handcrafted)
midi_loader = MidiLoader(midi_data, batch_size=1, shuffle=True)

In [33]:
dataiter = iter(midi_loader)
midi_vectors, midi_labels = dataiter.next()

In [37]:
print midi_vectors, midi_labels



Columns 0 to 10 
    57 -10000 -10000 -10000     41 -10000 -10000 -10000 -10000 -10000 -10000

Columns 11 to 21 
-10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000     83 -10000

Columns 22 to 32 
-10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000     79

Columns 33 to 43 
-10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000

Columns 44 to 54 
-10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000

Columns 55 to 65 
-10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000     60 -10000

Columns 66 to 76 
-10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000

Columns 77 to 87 
-10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000

Columns 88 to 98 
-10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000     76 -10000 -10000

Columns 99 to 109 
-10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000 -10000

Columns 110 to 120 
-10000 -10000 -100

In [19]:
test = torch.Tensor([[1., 2., 3.], [4., 1., 2.5]])
sorted_t, sorted_i = torch.sort(test.view(-1))
out_test = test.view(-1)
out_test.zero_()
out_test[sorted_i[-3:]] = 1
print test.view(2, 3)


 0  0  1
 1  0  1
[torch.FloatTensor of size 2x3]

